In [1]:
%matplotlib inline
import sys,os
import matplotlib.pyplot as plt
import torch
import spartan2.spartan as st

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] ="0"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
st.config(st.engine.SINGLEMACHINE)

In [3]:
# load time data
data = st.loadTensor(name = "example_time.tensor", path = "inputData/", col_types = [float, float, float], hasvalue=True)

Info: Tensor is loaded
           ----------------------
             attr     |	2
             values   |	True
             nonzeros |	36000



In [4]:
time_series = data.toTimeseries(attrlabels=['ecg_low', 'ecg_high'])

In [5]:
time_series.attrlists.shape

(2, 36000)

In [6]:
# create a beatgan model

ad_model = st.anomaly_detection.create(time_series, st.ad_policy.BEATGAN, "my_beatgan_model")

In [7]:
### param for train model
param_rnn = {
    'network':'CNN',
    'net_type':'gru',
    'layers':1,
    'seq_len': 64,
    'stride': 32 ,
    'input_size':2,
    'hidden_size':100,
    'rep_size':20,
    'batch_size':64,
    'max_epoch':5,
    'lr': 0.01,
    'lambda':1
}

In [8]:
ad_model.init_model(param_rnn,device)

[Conv1d(2, 32, kernel_size=(4,), stride=(2,), padding=(1,), bias=False), LeakyReLU(negative_slope=0.2, inplace=True), Conv1d(32, 64, kernel_size=(4,), stride=(2,), padding=(1,), bias=False), BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), LeakyReLU(negative_slope=0.2, inplace=True), Conv1d(64, 128, kernel_size=(4,), stride=(2,), padding=(1,), bias=False), BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), LeakyReLU(negative_slope=0.2, inplace=True), Conv1d(128, 256, kernel_size=(4,), stride=(2,), padding=(1,), bias=False), BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), LeakyReLU(negative_slope=0.2, inplace=True), Conv1d(256, 20, kernel_size=(4,), stride=(1,), bias=False)]
[ConvTranspose1d(20, 256, kernel_size=(4,), stride=(1,), bias=False), BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(inplace=True), ConvTranspose1d(256, 128, kernel_size=(4,), stri

In [9]:
ad_model.train(time_series)

  0%|          | 0/5 [00:00<?, ?it/s]

0:loss_g(rec/adv):0.1739777773618698/0.8265378475189209,loss_d(real/fake):0.8652509450912476/0.6067827939987183


 20%|██        | 1/5 [00:04<00:17,  4.32s/it]

0:loss_g(rec/adv):0.10077608376741409/0.2606324851512909,loss_d(real/fake):0.1959189474582672/0.009739088825881481


 40%|████      | 2/5 [00:08<00:12,  4.29s/it]

...

 60%|██████    | 3/5 [00:12<00:08,  4.26s/it]

0:loss_g(rec/adv):0.0940466821193695/0.21352529525756836,loss_d(real/fake):0.09540962427854538/0.0011823782697319984


 80%|████████  | 4/5 [00:16<00:04,  4.21s/it]

0:loss_g(rec/adv):0.0461902916431427/0.14362813532352448,loss_d(real/fake):0.0073326947167515755/0.489151269197464


100%|██████████| 5/5 [00:20<00:00,  4.19s/it]


In [10]:
ad_model.test(time_series)

[0.12642826 0.19472861 0.5964342  ... 0.08249463 0.10040393 0.10824323]


array([0.12642826, 0.19472861, 0.5964342 , ..., 0.08249463, 0.10040393, 0.10824323], dtype=float32)

In [11]:
ad_model.export("/Users/zhoubin/test/my_beatgan.pth")

In [12]:
## param for run
param_run = {
    'model_path':"/Users/zhoubin/test/my_beatgan.pth",
    'network':'CNN',
    'net_type':'gru',
    'layers':1,
    'seq_len': 64,
    'stride': 32 ,
    'input_size':2,
    'hidden_size':100,
    'rep_size':20,
    'batch_size':64,
    'max_epoch':5,
    'lr': 0.01,
    'lambda':1
}
### run model (load form model_path)
beatgan,res=ad_model.run(None,time_series,param_run,device)

[Conv1d(2, 32, kernel_size=(4,), stride=(2,), padding=(1,), bias=False), LeakyReLU(negative_slope=0.2, inplace=True), Conv1d(32, 64, kernel_size=(4,), stride=(2,), padding=(1,), bias=False), BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), LeakyReLU(negative_slope=0.2, inplace=True), Conv1d(64, 128, kernel_size=(4,), stride=(2,), padding=(1,), bias=False), BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), LeakyReLU(negative_slope=0.2, inplace=True), Conv1d(128, 256, kernel_size=(4,), stride=(2,), padding=(1,), bias=False), BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), LeakyReLU(negative_slope=0.2, inplace=True), Conv1d(256, 20, kernel_size=(4,), stride=(1,), bias=False)]
[ConvTranspose1d(20, 256, kernel_size=(4,), stride=(1,), bias=False), BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(inplace=True), ConvTranspose1d(256, 128, kernel_size=(4,), stri

In [13]:
### run model from cache
ad_model.run(beatgan,time_series,param_run,device)

[0.12642826 0.19472861 0.5964342  ... 0.08249463 0.10040393 0.10824323]
